In [4]:
import os
import sys
import pandas as pd
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import shutil
import subprocess

Divide COVID data into train, test, and val.

COVID-CT-master/Data-split/COVID/ has 3 files, containing the filenames of images located on COVID-CT-master/Images-processed/CT_COVID/.

    testCT_COVID
    trainCT_COVID
    valCT_COVID

We iterate through the each of the file and copy the images to

    dataset-git/train/covid
    dataset-git/test/covid
    dataset-git/val/covid



In [36]:
datasets = ['train', 'test', 'val']
data_covid = "COVID-CT-master/Data-split/COVID/"
images_covid = "COVID-CT-master/Images-processed/CT_COVID/"

for group in datasets:
    with open(data_covid+group+"CT_COVID.txt", 'r') as file:
        for cnt, line in enumerate(file):
            line = line.strip()
            shutil.copyfile(images_covid+line, 'revised-git/'+group+'/covid/' + line)

Divide NonCOVID data into train, test, and val.

COVID-CT-master/Data-split/NonCOVID/ has 3 files, containing the filenames of images located on COVID-CT-master/Images-processed/CT_NonCOVID/.
- testCT_NonCOVID
- trainCT_NonCOVID
- valCT_NonCOVID


We iterate through the each of the file and copy the images to
- dataset-git/train/noncovid
- dataset-git/test/noncovid
- dataset-git/val/noncovid

In [37]:
datasets = ['train', 'test', 'val']
data_noncovid = "COVID-CT-master/Data-split/NonCOVID/"
images_noncovid = "COVID-CT-master/Images-processed/CT_NonCOVID/"

for group in datasets:
    with open(data_noncovid+group+"CT_NonCOVID.txt", 'r') as file:
        for cnt, line in enumerate(file):
            line = line.strip()
            shutil.copyfile(images_noncovid+line, 'revised-git/'+group+'/noncovid/' + line)


Preprocessing images from kaggle

- List directories from covid, healthy, and others
- Set label covid = 1, healthy = 0, others = 0 for each directory (patient)
- Concatenate health and others as noncovid
- Build dataset from covid and noncovid
- Randomly divide dataset by train and test (test = 0.2)
- Add the images of the patients belonging to train to trainset
- Add the images of the patients belonging to test to testset

Copy images to dataset-kaggle:
- dataset-kaggle/train/covid
- dataset-kaggle/test/covid
- dataset-kaggle/train/noncovid
- dataset-kaggle/test/noncovid

In [10]:
import numpy as np
from sklearn.model_selection import train_test_split

dir_covid = 'new_sars_cov/Covid'
dir_healthy = 'new_sars_cov/Healthy'
dir_others = 'new_sars_cov/Others'

    
patients_covid = []
covid_labels = []
for filename in os.listdir(dir_covid):
    path = os.path.join(dir_covid, filename).replace("\\", r"/")
    patients_covid.append(path)
    covid_labels.append(1)
    
print(len(patients_covid))

patients_healthy = []
healthy_labels = []
for filename in os.listdir(dir_healthy):
    path = os.path.join(dir_healthy, filename).replace("\\", r"/")
    patients_healthy.append(path)
    healthy_labels.append(0)

print(len(patients_healthy))    
    
patients_others = []
others_labels = []
for filename in os.listdir(dir_others):
    path = os.path.join(dir_others, filename).replace("\\", r"/")
    patients_others.append(path)
    others_labels.append(0)

print(len(patients_others))  

80
50
80


In [11]:
dataset = np.concatenate((np.array(patients_covid), np.array(patients_healthy), np.array(patients_others)))
labels = np.concatenate((np.array(covid_labels), np.array(healthy_labels), np.array(others_labels)))

In [12]:
X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=0.2, random_state=42)

In [27]:
#read train images

final_train_labels = []
final_train_images = []
for id, dir in enumerate(X_train):
    label = y_train[id]
    for root, dirs, files in os.walk(dir):
        for name in files:
            path = os.path.join(root, name).replace("\\", r"/")
            if os.path.isfile(path):
                final_train_images.append(path)
                final_train_labels.append(label)
                

print(len(final_train_images))
print(len(final_train_labels))

3348
3348


In [28]:
#read test images

final_test_labels = []
final_test_images = []
for id, dir in enumerate(X_test):
    label = y_test[id]
    for root, dirs, files in os.walk(dir):
        for name in files:
            path = os.path.join(root, name).replace("\\", r"/")
            if os.path.isfile(path):
                final_test_images.append(path)
                final_test_labels.append(label)
                

print(len(final_test_images))
print(len(final_test_labels))

824
824


Iterate through the train files and copy to 

- revised-kaggle/train/covid if label is 1 
- revised-kaggle/train/noncovid if label is 0

In [34]:
for id,file in enumerate(final_train_images):
    print(file)
    label = final_train_labels[id]
    filename = file.split("/")[3]

    if(label==0):
        shutil.copy2(file, 'revised-kaggle/train/noncovid/' +str(id)+".png")
    else:
        shutil.copy2(file, 'revised-kaggle/train/covid/' +str(id)+".png")
    
            

new_sars_cov/Others/Patient (28)/Captura de Tela 2020-05-30 +ás 12.34.27.png
new_sars_cov/Others/Patient (28)/Captura de Tela 2020-05-30 +ás 12.34.32.png
new_sars_cov/Others/Patient (28)/Captura de Tela 2020-05-30 +ás 12.34.38.png
new_sars_cov/Others/Patient (28)/Captura de Tela 2020-05-30 +ás 12.34.45.png
new_sars_cov/Others/Patient (28)/Captura de Tela 2020-05-30 +ás 12.34.54.png
new_sars_cov/Others/Patient (28)/Captura de Tela 2020-05-30 +ás 12.35.02.png
new_sars_cov/Others/Patient (28)/Captura de Tela 2020-05-30 +ás 12.35.12.png
new_sars_cov/Others/Patient (28)/Captura de Tela 2020-05-30 +ás 12.35.21.png
new_sars_cov/Others/Patient (28)/Captura de Tela 2020-05-30 +ás 12.35.29.png
new_sars_cov/Others/Patient (28)/Captura de Tela 2020-05-30 +ás 12.35.38.png
new_sars_cov/Others/Patient (28)/Captura de Tela 2020-05-30 +ás 12.35.47.png
new_sars_cov/Others/Patient (28)/Captura de Tela 2020-05-30 +ás 12.35.55.png
new_sars_cov/Others/Patient (28)/Captura de Tela 2020-05-30 +ás 12.36.26.png

new_sars_cov/Covid/Patient (44)/AnotaGÇí+¦o 2020-04-29 165253.png
new_sars_cov/Covid/Patient (31)/Captura de Tela 2020-06-02 +ás 00.20.42.png
new_sars_cov/Covid/Patient (31)/Captura de Tela 2020-06-02 +ás 00.20.50.png
new_sars_cov/Covid/Patient (31)/Captura de Tela 2020-06-02 +ás 00.20.58.png
new_sars_cov/Covid/Patient (31)/Captura de Tela 2020-06-02 +ás 00.21.05.png
new_sars_cov/Covid/Patient (31)/Captura de Tela 2020-06-02 +ás 00.21.13.png
new_sars_cov/Covid/Patient (31)/Captura de Tela 2020-06-02 +ás 00.21.21.png
new_sars_cov/Covid/Patient (31)/Captura de Tela 2020-06-02 +ás 00.21.28.png
new_sars_cov/Covid/Patient (31)/Captura de Tela 2020-06-02 +ás 00.21.36.png
new_sars_cov/Covid/Patient (31)/Captura de Tela 2020-06-02 +ás 00.21.43.png
new_sars_cov/Covid/Patient (31)/Captura de Tela 2020-06-02 +ás 00.21.55.png
new_sars_cov/Covid/Patient (31)/Captura de Tela 2020-06-02 +ás 00.22.04.png
new_sars_cov/Covid/Patient (31)/Captura de Tela 2020-06-02 +ás 00.22.18.png
new_sars_cov/Covid/Pat

new_sars_cov/Covid/Patient (69)/12.png
new_sars_cov/Covid/Patient (69)/13.png
new_sars_cov/Covid/Patient (69)/14.png
new_sars_cov/Covid/Patient (69)/15.png
new_sars_cov/Covid/Patient (69)/16.png
new_sars_cov/Covid/Patient (69)/17.png
new_sars_cov/Covid/Patient (69)/18.png
new_sars_cov/Covid/Patient (69)/19.png
new_sars_cov/Covid/Patient (69)/2.png
new_sars_cov/Covid/Patient (69)/20.png
new_sars_cov/Covid/Patient (69)/21.png
new_sars_cov/Covid/Patient (69)/22.png
new_sars_cov/Covid/Patient (69)/23.png
new_sars_cov/Covid/Patient (69)/24.png
new_sars_cov/Covid/Patient (69)/25.png
new_sars_cov/Covid/Patient (69)/3.png
new_sars_cov/Covid/Patient (69)/4.png
new_sars_cov/Covid/Patient (69)/5.png
new_sars_cov/Covid/Patient (69)/6.png
new_sars_cov/Covid/Patient (69)/7.png
new_sars_cov/Covid/Patient (69)/8.png
new_sars_cov/Covid/Patient (69)/9.png
new_sars_cov/Covid/Patient (14)/AnotaGÇí+åo 2020-05-05 205643.png
new_sars_cov/Covid/Patient (14)/AnotaGÇí+åo 2020-05-05 205657.png
new_sars_cov/Covid

new_sars_cov/Covid/Patient (9)/AnotaGÇí+¦o 2020-05-05 143752.png
new_sars_cov/Covid/Patient (9)/AnotaGÇí+¦o 2020-05-05 143807.png
new_sars_cov/Covid/Patient (9)/AnotaGÇí+¦o 2020-05-05 143823.png
new_sars_cov/Covid/Patient (9)/AnotaGÇí+¦o 2020-05-05 143841.png
new_sars_cov/Covid/Patient (9)/AnotaGÇí+¦o 2020-05-05 143903.png
new_sars_cov/Covid/Patient (9)/AnotaGÇí+¦o 2020-05-05 143916.png
new_sars_cov/Covid/Patient (9)/AnotaGÇí+¦o 2020-05-05 143933.png
new_sars_cov/Covid/Patient (9)/AnotaGÇí+¦o 2020-05-05 144951.png
new_sars_cov/Covid/Patient (9)/AnotaGÇí+¦o 2020-05-05 145011.png
new_sars_cov/Covid/Patient (9)/AnotaGÇí+¦o 2020-05-05 145025.png
new_sars_cov/Covid/Patient (9)/AnotaGÇí+¦o 2020-05-05 145038.png
new_sars_cov/Covid/Patient (9)/AnotaGÇí+¦o 2020-05-05 145053.png
new_sars_cov/Covid/Patient (9)/AnotaGÇí+¦o 2020-05-05 145108.png
new_sars_cov/Covid/Patient (9)/AnotaGÇí+¦o 2020-05-05 145124.png
new_sars_cov/Others/Patient (78)/Captura de Tela 2020-05-30 +ás 05.14.35.png
new_sars_cov/

new_sars_cov/Covid/Patient (35)/AnotaGÇí+¦o 2020-05-04 205434.png
new_sars_cov/Covid/Patient (35)/AnotaGÇí+¦o 2020-05-04 205446.png
new_sars_cov/Covid/Patient (35)/AnotaGÇí+¦o 2020-05-04 205500.png
new_sars_cov/Covid/Patient (35)/AnotaGÇí+¦o 2020-05-04 205512.png
new_sars_cov/Covid/Patient (35)/AnotaGÇí+¦o 2020-05-04 205524.png
new_sars_cov/Covid/Patient (35)/AnotaGÇí+¦o 2020-05-04 205538.png
new_sars_cov/Covid/Patient (35)/AnotaGÇí+¦o 2020-05-04 205551.png
new_sars_cov/Covid/Patient (35)/AnotaGÇí+¦o 2020-05-04 205606.png
new_sars_cov/Covid/Patient (35)/AnotaGÇí+¦o 2020-05-04 205617.png
new_sars_cov/Covid/Patient (35)/AnotaGÇí+¦o 2020-05-04 205745.png
new_sars_cov/Covid/Patient (35)/AnotaGÇí+¦o 2020-05-04 205801.png
new_sars_cov/Covid/Patient (35)/AnotaGÇí+¦o 2020-05-04 205819.png
new_sars_cov/Covid/Patient (35)/AnotaGÇí+¦o 2020-05-04 205841.png
new_sars_cov/Covid/Patient (35)/AnotaGÇí+¦o 2020-05-04 205900.png
new_sars_cov/Covid/Patient (35)/AnotaGÇí+¦o 2020-05-04 205915.png
new_sars_c

...

new_sars_cov/Healthy/Patient (34)/Captura de Tela 2020-06-02 +ás 16.10.01.png
new_sars_cov/Healthy/Patient (34)/Captura de Tela 2020-06-02 +ás 16.10.08.png
new_sars_cov/Healthy/Patient (34)/Captura de Tela 2020-06-02 +ás 16.10.21.png
new_sars_cov/Healthy/Patient (34)/Captura de Tela 2020-06-02 +ás 16.10.34.png
new_sars_cov/Healthy/Patient (34)/Captura de Tela 2020-06-02 +ás 16.10.41.png
new_sars_cov/Healthy/Patient (34)/Captura de Tela 2020-06-02 +ás 16.10.49.png
new_sars_cov/Healthy/Patient (34)/Captura de Tela 2020-06-02 +ás 16.10.56.png
new_sars_cov/Healthy/Patient (34)/Captura de Tela 2020-06-02 +ás 16.11.07.png
new_sars_cov/Covid/Patient (55)/Captura de Tela 2020-06-02 +ás 00.15.02.png
new_sars_cov/Covid/Patient (55)/Captura de Tela 2020-06-02 +ás 00.15.12.png
new_sars_cov/Covid/Patient (55)/Captura de Tela 2020-06-02 +ás 00.15.19.png
new_sars_cov/Covid/Patient (55)/Captura de Tela 2020-06-02 +ás 00.15.25.png
new_sars_cov/Covid/Patient (55)/Captura de Tela 2020-06-02 +ás 00.15.31.

new_sars_cov/Covid/Patient (29)/AnotaGÇí+¦o 2020-05-05 135302.png
new_sars_cov/Covid/Patient (29)/AnotaGÇí+¦o 2020-05-05 135316.png
new_sars_cov/Covid/Patient (61)/AnotaGÇí+åo 2020-04-29 170003.png
new_sars_cov/Covid/Patient (61)/AnotaGÇí+åo 2020-04-29 170135.png
new_sars_cov/Covid/Patient (61)/AnotaGÇí+åo 2020-04-29 170206.png
new_sars_cov/Covid/Patient (61)/AnotaGÇí+åo 2020-04-29 170224.png
new_sars_cov/Covid/Patient (61)/AnotaGÇí+åo 2020-04-29 170248.png
new_sars_cov/Covid/Patient (61)/AnotaGÇí+åo 2020-04-29 170309.png
new_sars_cov/Covid/Patient (61)/AnotaGÇí+åo 2020-04-29 170332.png
new_sars_cov/Covid/Patient (61)/AnotaGÇí+åo 2020-04-29 170357.png
new_sars_cov/Covid/Patient (61)/AnotaGÇí+åo 2020-04-29 170429.png
new_sars_cov/Covid/Patient (61)/AnotaGÇí+åo 2020-04-29 170449.png
new_sars_cov/Covid/Patient (61)/AnotaGÇí+åo 2020-04-29 170510.png
new_sars_cov/Covid/Patient (61)/AnotaGÇí+åo 2020-04-29 170532.png
new_sars_cov/Covid/Patient (61)/AnotaGÇí+åo 2020-04-29 170615.png
new_sars_c

new_sars_cov/Covid/Patient (57)/AnotaGÇí+¦o 2020-04-28 140619.png
new_sars_cov/Covid/Patient (57)/AnotaGÇí+¦o 2020-04-28 140758.png
new_sars_cov/Covid/Patient (57)/AnotaGÇí+¦o 2020-04-28 141030.png
new_sars_cov/Covid/Patient (57)/AnotaGÇí+¦o 2020-04-28 141055.png
new_sars_cov/Covid/Patient (57)/AnotaGÇí+¦o 2020-04-28 141110.png
new_sars_cov/Covid/Patient (57)/AnotaGÇí+¦o 2020-04-28 141326.png
new_sars_cov/Covid/Patient (57)/AnotaGÇí+¦o 2020-04-28 141358.png
new_sars_cov/Covid/Patient (57)/AnotaGÇí+¦o 2020-04-28 141430.png
new_sars_cov/Covid/Patient (57)/AnotaGÇí+¦o 2020-04-28 141453.png
new_sars_cov/Covid/Patient (57)/AnotaGÇí+¦o 2020-04-28 141515.png
new_sars_cov/Covid/Patient (57)/AnotaGÇí+¦o 2020-04-28 141538.png
new_sars_cov/Covid/Patient (57)/AnotaGÇí+¦o 2020-04-28 141607.png
new_sars_cov/Covid/Patient (57)/AnotaGÇí+¦o 2020-04-28 141627.png
new_sars_cov/Covid/Patient (57)/AnotaGÇí+¦o 2020-04-28 141650.png
new_sars_cov/Covid/Patient (57)/AnotaGÇí+¦o 2020-04-28 141716.png
new_sars_c

new_sars_cov/Healthy/Patient (16)/Captura de Tela 2020-05-30 +ás 11.16.48.png
new_sars_cov/Healthy/Patient (16)/Captura de Tela 2020-05-30 +ás 11.16.58.png
new_sars_cov/Healthy/Patient (16)/Captura de Tela 2020-05-30 +ás 11.17.05.png
new_sars_cov/Healthy/Patient (16)/Captura de Tela 2020-05-30 +ás 11.17.11.png
new_sars_cov/Healthy/Patient (16)/Captura de Tela 2020-05-30 +ás 11.17.18.png
new_sars_cov/Covid/Patient (77)/1.png
new_sars_cov/Covid/Patient (77)/10.png
new_sars_cov/Covid/Patient (77)/11.png
new_sars_cov/Covid/Patient (77)/12.png
new_sars_cov/Covid/Patient (77)/13.png
new_sars_cov/Covid/Patient (77)/14.png
new_sars_cov/Covid/Patient (77)/15.png
new_sars_cov/Covid/Patient (77)/16.png
new_sars_cov/Covid/Patient (77)/17.png
new_sars_cov/Covid/Patient (77)/18.png
new_sars_cov/Covid/Patient (77)/19.png
new_sars_cov/Covid/Patient (77)/2.png
new_sars_cov/Covid/Patient (77)/20.png
new_sars_cov/Covid/Patient (77)/21.png
new_sars_cov/Covid/Patient (77)/22.png
new_sars_cov/Covid/Patient 

new_sars_cov/Covid/Patient (22)/AnotaGÇí+¦o 2020-04-29 191251.png
new_sars_cov/Covid/Patient (22)/AnotaGÇí+¦o 2020-04-29 191310.png
new_sars_cov/Covid/Patient (22)/AnotaGÇí+¦o 2020-04-29 191341.png
new_sars_cov/Covid/Patient (22)/AnotaGÇí+¦o 2020-04-29 191403.png
new_sars_cov/Healthy/Patient (20)/Captura de Tela 2020-06-01 +ás 20.23.48.png
new_sars_cov/Healthy/Patient (20)/Captura de Tela 2020-06-01 +ás 20.56.24.png
new_sars_cov/Healthy/Patient (20)/Captura de Tela 2020-06-01 +ás 20.56.34.png
new_sars_cov/Healthy/Patient (20)/Captura de Tela 2020-06-01 +ás 20.56.41.png
new_sars_cov/Healthy/Patient (20)/Captura de Tela 2020-06-01 +ás 20.56.48.png
new_sars_cov/Healthy/Patient (20)/Captura de Tela 2020-06-01 +ás 20.56.56.png
new_sars_cov/Healthy/Patient (20)/Captura de Tela 2020-06-01 +ás 20.57.12.png
new_sars_cov/Healthy/Patient (20)/Captura de Tela 2020-06-01 +ás 20.57.22.png
new_sars_cov/Healthy/Patient (20)/Captura de Tela 2020-06-01 +ás 20.57.29.png
new_sars_cov/Healthy/Patient (20)/

Iterate through the test files and copy to

- revised-kaggle/test/covid if label is 1
- revised-kaggle/test/noncovid if label is 0



In [35]:
for id,file in enumerate(final_test_images):
    
    label = final_test_labels[id]
    filename = file.split("/")[3]
    print(str(id)+file+str(label))

    if(label==0):
        shutil.copyfile(file, 'revised-kaggle/test/noncovid/' +str(id)+".png")
    else:
        shutil.copyfile(file, 'revised-kaggle/test/covid/' +str(id)+".png")


0new_sars_cov/Covid/Patient (37)/AnotaGÇí+åo 2020-05-05 213439.png1
1new_sars_cov/Covid/Patient (37)/AnotaGÇí+åo 2020-05-05 213604.png1
2new_sars_cov/Covid/Patient (37)/AnotaGÇí+åo 2020-05-05 213616.png1
3new_sars_cov/Covid/Patient (37)/AnotaGÇí+åo 2020-05-05 213631.png1
4new_sars_cov/Covid/Patient (37)/AnotaGÇí+åo 2020-05-05 213643.png1
5new_sars_cov/Covid/Patient (37)/AnotaGÇí+åo 2020-05-05 213700.png1
6new_sars_cov/Covid/Patient (37)/AnotaGÇí+åo 2020-05-05 213712.png1
7new_sars_cov/Covid/Patient (37)/AnotaGÇí+åo 2020-05-05 213724.png1
8new_sars_cov/Covid/Patient (37)/AnotaGÇí+åo 2020-05-05 213750.png1
9new_sars_cov/Covid/Patient (37)/AnotaGÇí+åo 2020-05-05 213806.png1
10new_sars_cov/Covid/Patient (37)/AnotaGÇí+åo 2020-05-05 213819.png1
11new_sars_cov/Covid/Patient (37)/AnotaGÇí+åo 2020-05-05 213833.png1
12new_sars_cov/Covid/Patient (37)/AnotaGÇí+åo 2020-05-05 213852.png1
13new_sars_cov/Covid/Patient (37)/AnotaGÇí+åo 2020-05-05 213907.png1
14new_sars_cov/Covid/Patient (37)/AnotaGÇí+å

144new_sars_cov/Others/Patient (57)/Captura de Tela 2020-05-30 +ás 04.33.03.png0
145new_sars_cov/Others/Patient (57)/Captura de Tela 2020-05-30 +ás 04.33.19.png0
146new_sars_cov/Others/Patient (57)/Captura de Tela 2020-05-30 +ás 04.33.36.png0
147new_sars_cov/Others/Patient (57)/Captura de Tela 2020-05-30 +ás 04.33.53.png0
148new_sars_cov/Others/Patient (57)/Captura de Tela 2020-05-30 +ás 04.34.12.png0
149new_sars_cov/Others/Patient (57)/Captura de Tela 2020-05-30 +ás 04.34.28.png0
150new_sars_cov/Others/Patient (57)/Captura de Tela 2020-05-30 +ás 04.34.45.png0
151new_sars_cov/Others/Patient (57)/Captura de Tela 2020-05-30 +ás 04.35.01.png0
152new_sars_cov/Others/Patient (57)/Captura de Tela 2020-05-30 +ás 04.35.20.png0
153new_sars_cov/Others/Patient (57)/Captura de Tela 2020-05-30 +ás 04.35.42.png0
154new_sars_cov/Covid/Patient (18)/AnotaGÇí+¦o 2020-04-29 203229.png1
155new_sars_cov/Covid/Patient (18)/AnotaGÇí+¦o 2020-04-29 203321.png1
156new_sars_cov/Covid/Patient (18)/AnotaGÇí+¦o 202

254new_sars_cov/Covid/Patient (23)/Captura de Tela 2020-06-02 +ás 09.36.33.png1
255new_sars_cov/Covid/Patient (23)/Captura de Tela 2020-06-02 +ás 09.37.12.png1
256new_sars_cov/Covid/Patient (23)/Captura de Tela 2020-06-02 +ás 09.37.18.png1
257new_sars_cov/Covid/Patient (23)/Captura de Tela 2020-06-02 +ás 09.37.24.png1
258new_sars_cov/Covid/Patient (23)/Captura de Tela 2020-06-02 +ás 09.37.30.png1
259new_sars_cov/Covid/Patient (23)/Captura de Tela 2020-06-02 +ás 09.37.40.png1
260new_sars_cov/Covid/Patient (23)/Captura de Tela 2020-06-02 +ás 09.38.17.png1
261new_sars_cov/Covid/Patient (23)/Captura de Tela 2020-06-02 +ás 09.38.24.png1
262new_sars_cov/Covid/Patient (23)/Captura de Tela 2020-06-02 +ás 09.38.31.png1
263new_sars_cov/Covid/Patient (23)/Captura de Tela 2020-06-02 +ás 09.38.37.png1
264new_sars_cov/Covid/Patient (23)/Captura de Tela 2020-06-02 +ás 09.38.43.png1
265new_sars_cov/Covid/Patient (76)/1.png1
266new_sars_cov/Covid/Patient (76)/10.png1
267new_sars_cov/Covid/Patient (76)/

377new_sars_cov/Covid/Patient (78)/15.png1
378new_sars_cov/Covid/Patient (78)/16.png1
379new_sars_cov/Covid/Patient (78)/17.png1
380new_sars_cov/Covid/Patient (78)/18.png1
381new_sars_cov/Covid/Patient (78)/19.png1
382new_sars_cov/Covid/Patient (78)/2.png1
383new_sars_cov/Covid/Patient (78)/20.png1
384new_sars_cov/Covid/Patient (78)/21.png1
385new_sars_cov/Covid/Patient (78)/22.png1
386new_sars_cov/Covid/Patient (78)/23.png1
387new_sars_cov/Covid/Patient (78)/24.png1
388new_sars_cov/Covid/Patient (78)/25.png1
389new_sars_cov/Covid/Patient (78)/26.png1
390new_sars_cov/Covid/Patient (78)/27.png1
391new_sars_cov/Covid/Patient (78)/28.png1
392new_sars_cov/Covid/Patient (78)/29.png1
393new_sars_cov/Covid/Patient (78)/3.png1
394new_sars_cov/Covid/Patient (78)/30.png1
395new_sars_cov/Covid/Patient (78)/31.png1
396new_sars_cov/Covid/Patient (78)/32.png1
397new_sars_cov/Covid/Patient (78)/33.png1
398new_sars_cov/Covid/Patient (78)/34.png1
399new_sars_cov/Covid/Patient (78)/35.png1
400new_sars_c

498new_sars_cov/Others/Patient (20)/Captura de Tela 2020-06-01 +ás 17.02.50.png0
499new_sars_cov/Others/Patient (20)/Captura de Tela 2020-06-01 +ás 17.02.55.png0
500new_sars_cov/Others/Patient (20)/Captura de Tela 2020-06-01 +ás 17.03.01.png0
501new_sars_cov/Others/Patient (20)/Captura de Tela 2020-06-01 +ás 17.03.07.png0
502new_sars_cov/Others/Patient (20)/Captura de Tela 2020-06-01 +ás 17.03.12.png0
503new_sars_cov/Others/Patient (20)/Captura de Tela 2020-06-01 +ás 17.03.18.png0
504new_sars_cov/Others/Patient (20)/Captura de Tela 2020-06-01 +ás 17.03.23.png0
505new_sars_cov/Others/Patient (20)/Captura de Tela 2020-06-01 +ás 17.03.29.png0
506new_sars_cov/Others/Patient (20)/Captura de Tela 2020-06-01 +ás 17.03.35.png0
507new_sars_cov/Others/Patient (20)/Captura de Tela 2020-06-01 +ás 17.03.40.png0
508new_sars_cov/Others/Patient (20)/Captura de Tela 2020-06-01 +ás 17.03.45.png0
509new_sars_cov/Others/Patient (20)/Captura de Tela 2020-06-01 +ás 17.03.51.png0
510new_sars_cov/Others/Patie

621new_sars_cov/Covid/Patient (72)/22.png1
622new_sars_cov/Covid/Patient (72)/23.png1
623new_sars_cov/Covid/Patient (72)/24.png1
624new_sars_cov/Covid/Patient (72)/25.png1
625new_sars_cov/Covid/Patient (72)/26.png1
626new_sars_cov/Covid/Patient (72)/27.png1
627new_sars_cov/Covid/Patient (72)/28.png1
628new_sars_cov/Covid/Patient (72)/29.png1
629new_sars_cov/Covid/Patient (72)/3.png1
630new_sars_cov/Covid/Patient (72)/30.png1
631new_sars_cov/Covid/Patient (72)/31.png1
632new_sars_cov/Covid/Patient (72)/32.png1
633new_sars_cov/Covid/Patient (72)/33.png1
634new_sars_cov/Covid/Patient (72)/34.png1
635new_sars_cov/Covid/Patient (72)/35.png1
636new_sars_cov/Covid/Patient (72)/36.png1
637new_sars_cov/Covid/Patient (72)/37.png1
638new_sars_cov/Covid/Patient (72)/38.png1
639new_sars_cov/Covid/Patient (72)/39.png1
640new_sars_cov/Covid/Patient (72)/4.png1
641new_sars_cov/Covid/Patient (72)/40.png1
642new_sars_cov/Covid/Patient (72)/5.png1
643new_sars_cov/Covid/Patient (72)/6.png1
644new_sars_cov

751new_sars_cov/Healthy/Patient (11)/Captura de Tela 2020-05-30 +ás 04.25.25.png0
752new_sars_cov/Healthy/Patient (11)/Captura de Tela 2020-05-30 +ás 04.25.42.png0
753new_sars_cov/Healthy/Patient (11)/Captura de Tela 2020-05-30 +ás 04.25.57.png0
754new_sars_cov/Healthy/Patient (11)/Captura de Tela 2020-05-30 +ás 04.26.13.png0
755new_sars_cov/Healthy/Patient (11)/Captura de Tela 2020-05-30 +ás 04.26.30.png0
756new_sars_cov/Healthy/Patient (11)/Captura de Tela 2020-05-30 +ás 04.26.46.png0
757new_sars_cov/Healthy/Patient (11)/Captura de Tela 2020-05-30 +ás 04.27.05.png0
758new_sars_cov/Healthy/Patient (11)/Captura de Tela 2020-05-30 +ás 04.27.22.png0
759new_sars_cov/Healthy/Patient (11)/Captura de Tela 2020-05-30 +ás 04.27.46.png0
760new_sars_cov/Healthy/Patient (11)/Captura de Tela 2020-05-30 +ás 04.28.01.png0
761new_sars_cov/Healthy/Patient (11)/Captura de Tela 2020-05-30 +ás 04.28.16.png0
762new_sars_cov/Others/Patient (36)/Captura de Tela 2020-06-01 +ás 21.45.10.png0
763new_sars_cov/O